### Import Modules

In [3]:
import json
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from numba import jit, float64
import time
import os

### Open and Interpolate Real World Data

In [4]:
def load_excel_data(directory):
    """
    Load all Excel files and their sheets from the specified directory into a list of dataframes.

    Parameters:
    directory (str): The path to the folder containing the Excel files.

    Returns:
    list: A list of pandas dataframes containing the data from each sheet in the Excel files.
    """
    # Initialize an empty list to store dataframes
    datasets = []

    # Iterate over each file in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(directory, filename)
            
            # Load the Excel file
            excel_file = pd.ExcelFile(file_path)
            
            # Iterate over each sheet in the Excel file
            for sheet_name in excel_file.sheet_names:
                # Load the sheet into a dataframe and append to the datasets list
                df = pd.read_excel(excel_file, sheet_name=sheet_name)
                datasets.append(df)

    return datasets

def save_datasets_to_excel(datasets, output_file):
    """
    Save a list of dataframes to a single Excel file with each dataframe in a separate sheet.

    Parameters:
    datasets (list): A list of tuples containing filename, sheet name, and dataframe.
    output_file (str): The path to the output Excel file.
    """
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        for i, (df) in enumerate(datasets):
            sheet_name_clean = f"Sheet_{i}"[:31]  # Excel sheet names must be <= 31 chars
            df.to_excel(writer, sheet_name=sheet_name_clean, index=False)



In [5]:
# Example usage of the function
directory = 'data/'
datasets = load_excel_data(directory)

In [6]:
# for i in range(len(datasets)):
#     plt.plot(datasets[i]['timestamp'], datasets[i]['trolley_position'], label='Trolley Position')
#     plt.plot(datasets[i]['timestamp'], datasets[i]['cable_length'], label='Cable Length')
#     plt.xlabel('Time')
#     plt.legend()
#     plt.show()

In [7]:
DURATION = 15  # duration in seconds
DT = 0.0001  # time increment in seconds
# Create a time array
time_array = np.arange(0, DURATION + DT, DT)
NUM_STEPS = len(time_array)

interpolated_datasets = []
for i in range (len(datasets)):
    new_trolley_position = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["trolley_position"]
    )
    new_cable_length = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["cable_length"]
    )
    new_sway_angle = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["sway_angle"]
    )
    new_trolley_motor_voltage = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["trolley_motor_voltage"]
    )
    new_hoist_motor_voltage = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["hoist_motor_voltage"]
    )

    interpolated_df = pd.DataFrame(
        {
            "time": time_array,
            "trolley_position": new_trolley_position,
            "cable_length": new_cable_length,
            "sway_angle": new_sway_angle,
            "trolley_motor_voltage": new_trolley_motor_voltage,
            "hoist_motor_voltage": new_hoist_motor_voltage,
        }
    )

    interpolated_datasets.append(interpolated_df)

In [8]:
# for i in range(len(interpolated_datasets)):
#     plt.plot(interpolated_datasets[i]['time'], interpolated_datasets[i]['trolley_position'], label='Trolley Position')
#     plt.plot(interpolated_datasets[i]['time'], interpolated_datasets[i]['cable_length'], label='Cable Length')
#     plt.xlabel('Time')
#     plt.legend()
#     plt.show()

In [9]:
# save_datasets_to_excel(interpolated_datasets, 'interpolated_datasets.xlsx')

### Open and Set Model Parameters

In [10]:
# Open the JSON file
with open("gantry_crane_parameters.json", "r") as file:
    data = json.load(file)

all_parameters = data["gantry_crane_system_model"]["parameters"]
measured_parameters = {}
approximated_parameters = {}

for parameter in all_parameters:
    value = all_parameters[parameter]["value"]
    unit = all_parameters[parameter]["unit"]
    description = all_parameters[parameter]["description"]

    if all_parameters[parameter]["measured"]:
        measured_parameters[parameter] = all_parameters[parameter]
    else:
        approximated_parameters[parameter] = all_parameters[parameter]

    # print(f"{parameter}: {value} {unit} ({description})")

##### Optimized Function to Calculate Squared Error

In [11]:
@jit(nopython=True)
def calculate_squared_errors(y1, y2):
    squared_errors = np.zeros(len(y1))
    for i in range(len(y1)):
        squared_errors[i] = (y1[i] - y2[i]) ** 2
    return squared_errors

##### Optimized Function to Calculate Sum Root Mean Squared Error (RMSE)

In [12]:
@jit(nopython=True)
def calculate_sum_root_mean_squared_errors(dataframe_header_, interpolated_array_, simulated_array_):
    sum_root_mean_squared_errors = 0.0
    for i in range (1, len(dataframe_header_)):
        y1 = interpolated_array_[i]
        y2 = simulated_array_[i]
        root_mean_squared_errors = np.sqrt(sum(calculate_squared_errors(y1, y2))/len(y1))
        sum_root_mean_squared_errors += root_mean_squared_errors
    return sum_root_mean_squared_errors

### Define Searching Range to Optimize Model Parameters

In [13]:
optimize_range = {
    "trolley_mass": (1.0, 5.0), # Done
    "trolley_damping_coefficient": (0.1, 10.0), # Done
    "cable_damping_coefficient": (0.1, 10.0), # Done
    "trolley_motor_rotator_inertia": (0.00001, 0.001), # Done
    "trolley_motor_damping_coefficient": (0.1, 10.0),
    "trolley_motor_back_emf_constant": (0.001, 0.1), # Done
    "trolley_motor_torque_constant": (0.01, 1.0), # Done
    "hoist_motor_rotator_inertia": (0.000001, 0.0001), # Done
    "hoist_motor_damping_coefficient": (0.1, 10.0),
    "hoist_motor_back_emf_constant": (0.01, 1.0), # Done
    "hoist_motor_torque_constant": (0.0001, 0.01),  # Done
}

### Declare Simulator Object

In [14]:
from model import Simulator
simulator = Simulator(DT, NUM_STEPS)

## Simulated Annealing

## Gradient Descent

### Define learning rate for each parameter

In [15]:
parameters_learning_rate = {}

for parameter in optimize_range:
    parameters_learning_rate[parameter] = min(optimize_range[parameter]) / 10

print("Learning rate:", parameters_learning_rate)

# Create random value for each parameter within the specified range
random_parameters = {}
for parameter in optimize_range:
    random_parameters[parameter] = np.random.uniform(
        optimize_range[parameter][0], optimize_range[parameter][1]
    )

print("Initial value:", random_parameters)

Learning rate: {'trolley_mass': 0.1, 'trolley_damping_coefficient': 0.01, 'cable_damping_coefficient': 0.01, 'trolley_motor_rotator_inertia': 1.0000000000000002e-06, 'trolley_motor_damping_coefficient': 0.01, 'trolley_motor_back_emf_constant': 0.0001, 'trolley_motor_torque_constant': 0.001, 'hoist_motor_rotator_inertia': 1e-07, 'hoist_motor_damping_coefficient': 0.01, 'hoist_motor_back_emf_constant': 0.001, 'hoist_motor_torque_constant': 1e-05}
Initial value: {'trolley_mass': 1.2608458049073938, 'trolley_damping_coefficient': 6.632568419460295, 'cable_damping_coefficient': 6.6717631972198825, 'trolley_motor_rotator_inertia': 0.00058743060930506, 'trolley_motor_damping_coefficient': 7.925188640423019, 'trolley_motor_back_emf_constant': 0.031757111381649654, 'trolley_motor_torque_constant': 0.1823490092584857, 'hoist_motor_rotator_inertia': 9.112077230302628e-06, 'hoist_motor_damping_coefficient': 0.2712427780755701, 'hoist_motor_back_emf_constant': 0.6840380649407936, 'hoist_motor_torqu

In [16]:
optimized_parameters = all_parameters

for parameter in random_parameters:
    optimized_parameters[parameter]["value"] = random_parameters[parameter]

old_parameter = optimized_parameters

In [17]:
interpolated_array = interpolated_datasets[0].values
dataframe_header = interpolated_datasets[0].columns

print("Interpolated array shape:", interpolated_array.shape)
print("Dataframe header:", dataframe_header)

Interpolated array shape: (150001, 6)
Dataframe header: Index(['time', 'trolley_position', 'cable_length', 'sway_angle',
       'trolley_motor_voltage', 'hoist_motor_voltage'],
      dtype='object')


: 

#### Optimize Model Parameters with Gradient Descent Algoritm

In [18]:
num_steps = len(interpolated_array[0])
input_voltages = {'trolley_motor_voltage': interpolated_array[4], 'hoist_motor_voltage': interpolated_array[5]}
variables_initial_conditions = {'x': interpolated_array[1][0], 'l': interpolated_array[2][0], 'theta': interpolated_array[3][0]}
epoch = 0
count = 0
epochs = []
total_sum_root_mean_squared_errors_array = []
while epoch <= 2000:
    steps = 0
    total_sum_root_mean_squared_errors = 0
    for parameter in optimize_range:
        simulator.simulate(optimized_parameters, input_voltages, variables_initial_conditions)
        simulated_array = simulator.get_results_for_optimization()

        sum_root_mean_squared_errors = calculate_sum_root_mean_squared_errors(
            dataframe_header, interpolated_array, simulated_array
        )

        # Calculate the cost function derivative relative to parameter value
        h = 1e-6
        optimized_parameters[parameter]["value"] += h
        simulator.simulate(optimized_parameters, input_voltages, variables_initial_conditions)
        simulated_array_h = simulator.get_results_for_optimization()

        sum_root_mean_squared_errors_h = calculate_sum_root_mean_squared_errors(
            dataframe_header, interpolated_array, simulated_array_h
        )

        derivative = (sum_root_mean_squared_errors_h - sum_root_mean_squared_errors) / h

        # print(derivative)

        steps = parameters_learning_rate[parameter] * derivative
        optimized_parameters[parameter]["steps"] = steps
        optimized_parameters[parameter]["value"] -= steps

        # print(end="\r")
        # print(
        #     "Parameter:",
        #     parameter,
        #     "| Value:",
        #     optimized_parameters[parameter]["value"],
        #     "| Loss:",
        #     sum_root_mean_squared_errors,
        #     "Step size:",
        #     steps,
        #     end="\r",
        # )

        total_sum_root_mean_squared_errors += sum_root_mean_squared_errors

    count = 0
    for parameter in optimize_range:
        if abs(optimized_parameters[parameter]["steps"]) < 1e-6:
            count += 1

    print(
            "Epoch:",
            epoch,
            "\t| Total loss:",
            round(total_sum_root_mean_squared_errors, 6),
            "\t| Converge count:",
            count,
            end="\r",
        )
    
    if count == len(optimized_parameters):
        break

    epochs.append(epoch)
    total_sum_root_mean_squared_errors_array.append(total_sum_root_mean_squared_errors)

    epoch += 1

In [ ]:
plt.plot(epochs, total_sum_root_mean_squared_errors_array)
plt.show()

In [ ]:
json_string = json.dumps(optimized_parameters, indent=4)

# Step 3: Write the JSON string to a file
with open("optimized_parameter.json", "w") as json_file:
    json_file.write(json_string)

for parameter in optimized_parameters:
    # print(parameter)
    if (
        old_parameter[parameter]["value"]
        != optimized_parameters[parameter]["value"]
    ):
        print("Old", parameter, "value:", old_parameter[parameter]["value"])
        print("Old", parameter, "value:", optimized_parameters[parameter]["value"])

### Simulate Optimized Parameters and Compare to Real Data

In [ ]:
num_steps = len(interpolated_array[0])
input_voltages = {
    "trolley_motor_voltage": interpolated_array[4],
    "hoist_motor_voltage": interpolated_array[5],
}
variables_initial_conditions = {
    "x": interpolated_array[1][0],
    "l": interpolated_array[2][0],
    "theta": interpolated_array[3][0],
}
simulator.simulate(all_parameters, input_voltages, variables_initial_conditions)

simulated_array = simulator.get_results_for_optimization()
for i in range(1, len(dataframe_header)):
    y1 = np.array(interpolated_array[i])
    y2 = np.array(simulated_array[i])

    plt.plot(time_array, y1, label=f"Interpolated {dataframe_header[i]}")
    plt.plot(time_array, y2, label=f"Simulated {dataframe_header[i]}")
    plt.legend()
    plt.show()